In [3]:
# package from Xilinx pynq
from pynq import Overlay
from pynq import allocate
import ipywidgets as widgets

In [4]:
#The FPGA overlay
ol = Overlay('./HVNMR.bit')

In [37]:
pulse_gen = ol.fpga_pulse_generator_0    # DACs  Pulse control unit
# start the pulse sequence
C_SEQUENCE_GENERATOR_EN = 0x0

# set the number of section
C_SET_NR_SECTIONS = 0x4

# write section select
C_SEL_SECTION = 0x8

# set section type
C_SET_SECTION_TYPE = 0xC

# set section duration
C_SET_DELAY = 0x10

# set section muxplexer
C_SET_MUX = 0x14

# set repetition start section pointer
C_SET_START_REPEAT_POINTER = 0x18

# set repetition end section pointer
C_SET_END_REPEAT_POINTER = 0x1C

# set cycle repetition number
C_SET_CYCLE_REPETITION_NUMBER = 0x20

# set experiment repetition number
C_SET_EXPERIMENT_REPETITION_NUMBER = 0x24

# channel 0 set phase
C_SET_PHASE_CH0 = 0x28

# channel 0 set frequency
C_SET_FREQUENCY_CH0 = 0x2C

# channel 1 set phase
C_SET_PHASE_CH1 = 0x30

# channel 1 set frequency
C_SET_FREQUENCY_CH1 = 0x34

# DDS reset
C_SET_RESETN_DDS = 0x38

# channel busy signal
C_GET_BUSY = 0x3C

# channel data ready signal
C_GET_DATA_READY = 0x40

# set number of DDS channel
C_GET_NR_DDS_CH = 0x44

# set memory depth
C_GET_MEM_DEPTH = 0x48

# set number of activity
C_GET_NR_ACTIVITY = 0x4C

# start
# C_START = 80   # 0x50

# write configuration data or experiment execution command
SetCommand = lambda command,value: pulse_gen.write(command,value)

# read data from pulse generator IP core
GetData = lambda command: pulse_gen.read(command)

def Sec_Config(sec,s_type,delay,mux,p0,f0,p1,f1,rstn):
        # choose section to configure
        SetCommand(C_SEL_SECTION,sec)
        # define section type
        SetCommand(C_SET_SECTION_TYPE,s_type)
        # define delay duration
        SetCommand(C_SET_DELAY,delay)
        # define multiplexer type
        SetCommand(C_SET_MUX,mux)
        # DDS channel 0 phase
        SetCommand(C_SET_PHASE_CH0,p0)
        # DDS channel 0 frequency
        SetCommand(C_SET_FREQUENCY_CH0,f0)
        # DDS channel 1 phase
        SetCommand(C_SET_PHASE_CH1,p1)
        # DDS channel 1 frequency
        SetCommand(C_SET_FREQUENCY_CH1,f1)
        # Reset
        SetCommand(C_SET_RESETN_DDS,rstn)
        
def Exp_Config():
    SetCommand(C_SET_NR_SECTIONS, section_nr)

    SetCommand(C_SET_START_REPEAT_POINTER, start_pointer)

    SetCommand(C_SET_END_REPEAT_POINTER, end_pointer)
    
    SetCommand(C_SET_CYCLE_REPETITION_NUMBER, cycle_repetition_number)
    
    SetCommand(C_SET_EXPERIMENT_REPETITION_NUMBER, experiment_repetition_number)


In [87]:
#! user setting the frequency 
f_ref = 0.1 # in MHz

In [88]:
# setting fpga nmr pulse generator
section_nr = 6
start_pointer = 2
end_pointer = 4
cycle_repetition_number = 1 
experiment_repetition_number =10000
Exp_Config()

t = 2.5e2                        # 1 us -- 250 MHz
TE = 3000*t                      # Echo Time
pi = 536870912                   # 180 degree, for 30 bits phase width uni parameter
f = 42949673                     
f = f * 4/ 10                    
fl = (f * 4/4)*f_ref
l_type  =  [0,0,0,0,0,0] #2:delay, 1:RX, 0:TX 
l_delay =  [2**15,2**31,2**31,2**31,2**31,2**31] 
l_mux   =  [0,0,0,0,0,0]
l_p0    =  [0,0,0,0,0,0]
l_f0    =  [f,f,f,f,f,f]
l_p1    =  [0,0,0,0,0,0]
l_f1    =  [fl,fl,fl,fl,fl,fl]
l_rstn  =  [1,1,1,1,1,1]

for i in range(0,len(l_delay)):
    l_delay[i] = int(l_delay[i])
    l_p0[i] = int(l_p0[i])
    l_f0[i] = int(l_f0[i])
    l_p1[i] = int(l_p1[i])
    l_f1[i] = int(l_f1[i])
    l_rstn[i] = int(l_rstn[i])
    
SetCommand(C_SEQUENCE_GENERATOR_EN,0)
for i in range(0,section_nr):
    Sec_Config(i,l_type[i],l_delay[i],l_mux[i],l_p0[i],l_f0[i],l_p1[i],l_f1[i],l_rstn[i])

# start testing
SetCommand(C_SEQUENCE_GENERATOR_EN,0)
SetCommand(C_SEQUENCE_GENERATOR_EN,1)

In [28]:
file_Debug_DAC_System = open("Debug_setup/Debug_DAC_System.jpg", "rb")
image = file_Debug_DAC_System.read()
widgets.Image(value=image,format='png',width=900,height=700,)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00H\x00H\x00\x00\xff\xe1\x00\x80Exif\x00\x00MM\x00…

In [11]:
file_Debug_DAC_Osci = open("Debug_setup/Debug_DAC_Osci.png", "rb")
image = file_Debug_DAC_Osci.read()
widgets.Image(value=image,format='png',width=900,height=700,)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x07\x80\x00\x00\x048\x08\x06\x00\x00\x00\xe8\xd3\xc1…

In [84]:
import numpy as np 
np.tanh(0.00001)


9.999999999666668e-06